## 🧱 3. Warehouse Modeling with SCD
 In this notebook, we:
 - Build dimension and fact tables in a star schema.
 - Apply SCD Type 1 and Type 2 logic to the `dim_product` table.
 - Save the modeled tables as optimized Parquet files.

### ⚙️ Setup & Spark Session

In [ ]:
!pip install pyspark==3.1.2 --quiet

from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .appName("Warehouse Modeling") \
    .getOrCreate()

warehouse_path = "/content/big_data_etl_project/4_data_warehouse/warehouse"
output_path = "/content/big_data_etl_project/5_scd_dimension_modeling"

### 📦 Step 1: Load Cleaned Parquet Data

In [ ]:
df_sales = spark.read.parquet(os.path.join(warehouse_path, "sales_logs"))
df_activity = spark.read.parquet(os.path.join(warehouse_path, "user_activity_logs"))
df_inventory = spark.read.parquet(os.path.join(warehouse_path, "inventory_events"))

df_sales.printSchema()
df_sales.show(2)

### 🧱 Step 2: Create Basic Dimensions

In [ ]:
from pyspark.sql.functions import col, monotonically_increasing_id

# -- dim_region --
dim_region = df_sales.select("region").distinct().withColumn("region_id", monotonically_increasing_id())

# -- dim_time --
from pyspark.sql.functions import year, month, dayofmonth

dim_time = df_sales.select("timestamp").distinct() \
    .withColumn("year", year("timestamp")) \
    .withColumn("month", month("timestamp")) \
    .withColumn("day", dayofmonth("timestamp")) \
    .withColumn("time_id", monotonically_increasing_id())

# -- dim_customer --
dim_customer = df_sales.select("customer_id").distinct().withColumn("customer_key", monotonically_increasing_id())

# Save basic dimensions
dim_region.write.mode("overwrite").parquet(os.path.join(output_path, "dim_region"))
dim_time.write.mode("overwrite").parquet(os.path.join(output_path, "dim_time"))
dim_customer.write.mode("overwrite").parquet(os.path.join(output_path, "dim_customer"))

print("✅ Saved dim_region, dim_time, dim_customer")

### 🔁 Step 3: SCD Type 1 – Overwrite Product Master

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

product_schema = StructType([
    StructField("product_id", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True)
])

product_master_data = [
    ("p1", "iPhone 13", "Electronics", 999.99),
    ("p2", "Airpods Pro", "Electronics", 249.99),
    ("p3", "Office Chair", "Furniture", 189.99)
]

df_product_master = spark.createDataFrame(product_master_data, schema=product_schema)
df_product_master.write.mode("overwrite").parquet(os.path.join(output_path, "dim_product_scd1"))

print("✅ Saved dim_product_scd1 (SCD Type 1)")

### 🕓 Step 4: SCD Type 2 – Product with History

In [ ]:
from pyspark.sql import Row
from datetime import datetime

history_data = [
    Row(product_id="p1", product_name="iPhone 12", category="Electronics", price=899.99,
        valid_from="2021-01-01", valid_to="2022-09-13", is_current=False),
    Row(product_id="p1", product_name="iPhone 13", category="Electronics", price=999.99,
        valid_from="2022-09-14", valid_to=None, is_current=True),
    Row(product_id="p3", product_name="Office Chair", category="Furniture", price=179.99,
        valid_from="2021-01-01", valid_to="2023-01-01", is_current=False),
    Row(product_id="p3", product_name="Office Chair", category="Furniture", price=189.99,
        valid_from="2023-01-02", valid_to=None, is_current=True)
]

df_product_scd2 = spark.createDataFrame(history_data)
df_product_scd2.write.mode("overwrite").parquet(os.path.join(output_path, "dim_product_scd2"))

print("✅ Saved dim_product_scd2 (SCD Type 2)")

### 🧮 Step 5: Create Enriched Fact Table

In [ ]:
from pyspark.sql.functions import expr

fact_sales_enriched = df_sales \
    .join(dim_region, on="region", how="left") \
    .join(dim_customer, on="customer_id", how="left") \
    .join(dim_time, on="timestamp", how="left") \
    .select(
        "timestamp", "customer_id", "product_id", "region", "price", "quantity", "payment_type",
        "region_id", "customer_key", "time_id"
    )

fact_sales_enriched.write.mode("overwrite").parquet(os.path.join(output_path, "fact_sales_enriched"))
print("✅ Saved enriched fact_sales table")

## ✅ Summary
 - Built dimensions: `region`, `time`, `customer`, and `product` (SCD1 & SCD2)
 - Created enriched `fact_sales` table with surrogate keys
 - Warehouse is now ready for analytics and ML pipeline development
